<a href="https://colab.research.google.com/github/bibekyess/dacon-hansol-deco-challenge/blob/main/pdf_combined_RAG_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
!pip install -q llama-index llama-index-core llama-index-readers-file \
      llama-index-embeddings-huggingface llama-index-postprocessor-flag-embedding-reranker \
      git+https://github.com/FlagOpen/FlagEmbedding.git llama-parse \
      llama-index-llms-vllm vllm


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.3 

In [ ]:
pip install peft bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DACON_wallpaper_contest_data

/content/drive/.shortcut-targets-by-id/1JIUWrdrv01QBIWHHKebD2f406jfGhSFu/DACON_wallpaper_contest_data


In [ ]:
!ls

best-one			orion-peft		test.csv
clean-files-for-retriever	orion_responses.json	test.gsheet
data				orion_submission.csv	third_submission.csv
demo_pretrained			outputs			train.csv
Domain_related_references	pre-processed-data	train.gsheet
fun_submission.csv		RAG-llama-index.ipynb	train-plus-sup-pdf-vector-index-storage
fun_submission_v2.csv		sample_submission.csv	train-vector-index-storage
good_responses.json		second_submission.csv	train-vector-index-storage-chunk-size-1295
good_responses_v2.json		submission.csv
good_responses_v3_till_56.json	summarized_output.json


In [ ]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.vllm import Vllm

In [ ]:
# os.environ["HF_HOME"] = "model/"

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3",
    pooling="mean" # "cls" is default
)

# llm = Vllm(
#     model="AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B",
#     tensor_parallel_size=1,
#     max_new_tokens=264,
#     dtype="half",
#     trust_remote_code=True,
#     vllm_kwargs={"swap_space": 1, "gpu_memory_utilization": 0.5},
# )

# Settings.llm = Vllm(model="dummy-model", api_url="lets trick")
Settings.llm = None # Lets go with dummy llm
Settings.embed_model = embed_model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [ ]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# The csv file is read in left-to-right and then top-to-down order

# check if storage already exists
PERSIST_DIR = "/content/drive/MyDrive/DACON_wallpaper_contest_data/train-plus-sup-pdf-vector-index-storage"
# if not os.path.exists(PERSIST_DIR):
#     # load the documents and create the index
#     documents = SimpleDirectoryReader("/content/drive/MyDrive/DACON_wallpaper_contest_data/data").load_data()
#     index = VectorStoreIndex.from_documents(
#         documents,
#         insert_batch_size=2
#     )

#     # store it for later
#     index.storage_context.persist(persist_dir=PERSIST_DIR)
# else:
# load the existing index
storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
index = load_index_from_storage(storage_context)


In [ ]:
# query_engine = index.as_query_engine()
# response = query_engine.query("면진장치가 뭐야?")
# # print(response.source_nodes)
# response.response

In [ ]:
# !pip install llama-index-postprocessor-flag-embedding-reranker


In [ ]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# build index
# index = VectorStoreIndex.from_documents(documents)
Settings.llm=None
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    # Choose one mode acccording to the need
    # https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes.html
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)




LLM is explicitly disabled. Using MockLLM.


In [ ]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=4,
    model="Dongjin-kr/ko-reranker",
)

raw_query_engine = index.as_query_engine(similarity_top_k=6, node_postprocessors=[reranker], verbose=True)

response_1 = raw_query_engine.query("면진장치가 뭐야?")
response_1


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Response(response='Context information is below.\n---------------------\nfile_path: /content/drive/MyDrive/DACON_wallpaper_contest_data/clean-files-for-retriever/good_train_long.csv\n\n건축구조, 면진장치에 사용되는 주요 기술은 무엇인가요? 면진장치가 뭐야?, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다. 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다. 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.\n\nfile_path: /content/drive/MyDrive/DACON_wallpaper_contest_data/clean-files-for-retriever/good_train_long.csv\n\n건축구조, 면진구조에 대해 좀 더 자세히 알 수 있을까요? 면진구조가 뭐야?, 면진구조란 면진 장치를 사용해 지반과 건물을 분리시키는 방법을 의미합니다. 면진구조는 지반과 건물을 분리시키기 위해 면진 장치를 사용하는 구조를 말합니다. 이 구조는 지진이나 지반의 이동으로 인한 건물의 피해를 최소화하기 위해 설계되었습니다. 면진구조는 지진으로 인한 건물의 피해를 최소화하는 데 도움을 주며, 안전성을 확보하기 위한 중요한 건축 기술 중 하나입니다. 면진구조란 지반과 건물을 분리시키기 위해 면진 장치를 사용하는 구조를 말합니다. 이

In [ ]:
context_list = []
for r in response_1.source_nodes:
  if r.score > 0:
      context_list.append(r.text)
context_list

['건축구조, 면진장치에 사용되는 주요 기술은 무엇인가요? 면진장치가 뭐야?, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다. 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다. 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.']

In [ ]:
## Uncomment this for new checkpoints
def get_prompt(question, prev_q="", raw_query_engine=raw_query_engine):
    # prev_q is a must needed for some questions like this: What is the biggest cause of plaster revision? And please tell me how to solve this.”
    INSTRUCTION_PROMPT_TEMPLATE = """\
    ### System:
    벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

    ### User:
    <보고서>
    {CONTEXT}
    </보고서>
    지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
    <질문>
    {QUESTION}
    </질문>

    ### Assistant:
    """
    RESPONSE_TEMPLATE = """\
    {ANSWER}

    """

    response_1 = raw_query_engine.query(question)

    context_list = []
    for r in response_1.source_nodes:
      # print(r.score)
      if r.score > 0:
          if r.score <= 8 and len(context_list) >= 1:
              pass
          else:
              context_list.append(r.text)

    # Special case when the follow up question is junk
    if len(context_list) == 0:
        response_2 = raw_query_engine.query(prev_q + " " + question)
        for r in response_1.source_nodes:
          if r.score > 0:
              context_list.append(r.text)

    context = prev_q + "\n\n".join(context_list + [question])

    return INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)

get_prompt("또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?")

'    ### System:\n    벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n    ### User:\n    <보고서>\n    마감재, 원목사이딩을 사용하는데 어떤 단점이 있을까요? 원목사이딩의 단점이 뭐야?, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다. 원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다. 원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다.\n\n또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?\n    </보고서>\n    지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.\n    <질문>\n    또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?\n    </질문>\n\n    ### Assistant:\n    '

In [ ]:
get_prompt("도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?")

'    ### System:\n    벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n    ### User:\n    <보고서>\n    시공, 도배 공사 이후에 어느 정도의 시간을 공간을 건조시켜야 하는 것이 적절한가요? 도배 후 건조는 얼마나 해야 해?, 도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다. 도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수 있습니다. 건조 기간 동안에는 온도와 습도에 유의해야 하며, 일주일이 지난 후에도 도배지가 완전히 건조되지 않았다면 전문가의 도움을 받아 진단받아야 합니다. 도배 후 도배지가 완전히 건조되기까지 최소 일주일이 걸릴 수 있습니다. 건조 기간 동안 온도와 습도에 유의하여, 제조사의 권장사항을 준수하는 것이 중요합니다. 만약 일주일이 지나도 완전히 건조되지 않는다면 시공사에게 상담을 요청하는 것이 좋습니다. 도배 후 도배지가 완전히 건조되기까지는 보통 최대 일주일의 시간이 필요합니다. 하지만 환경적 요인에 따라 건조 시간이 달라질 수 있으므로 습도와 온도를 고려하여 적정한 시간을 고려해야 합니다. 또한, 제조사의 권장 건조시간을 따르는 것이 중요하며, 도배작업이 완전히 마무리되기 전에는 벽면에 부담을 주지 말고 조심해야 합니다.\n\n도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?\n    </보고서>\n    지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.\n    <질문>\n    도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?\n    </질문>\n\n    ### Assistant:\n    '

In [ ]:
get_prompt("Explain more about this?")

'    ### System:\n    벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n    ### User:\n    <보고서>\n    Explain more about this?\n    </보고서>\n    지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.\n    <질문>\n    Explain more about this?\n    </질문>\n\n    ### Assistant:\n    '

In [ ]:
# QUESTION = "경량철골구조가 뭐야?"
# QUESTION = "면진장치가 뭐야?"

In [ ]:
# query_engine

In [ ]:
# response = query_engine.query(QUESTION)
# # print(response.source_nodes)
# response.response

### Similar thing can be done differently after retrieving the contexts.

In [ ]:
# contexts = retriever.retrieve(QUESTION)
# context_list = [n.get_content() for n in contexts]
# CONTEXT = "\n\n".join(context_list + [QUESTION])
# CONTEXT

In [ ]:
# pip install peft bitsandbytes accelerate

In [ ]:
# del model
# torch.empty_cache()

In [ ]:
from peft import AutoPeftModelForCausalLM
import torch
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/DACON_wallpaper_contest_data/best-one/checkpoint-2000",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    # load_in_4bit=True,
    quantization_config = bnb_config,
    device_map="auto" #"cuda" if low_cpu_mem_usage=False
)

merged_model = model.merge_and_unload()

config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
gogo

In [ ]:
from transformers import AutoTokenizer

model_id = "Edentns/DataVortexS-10.7B-dpo-v1.11" # Follows Alpaca Chat format
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, eos_token='###')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.eos_token

'###'

In [ ]:
from transformers import pipeline

text_pipe = pipeline("text-generation", merged_model, tokenizer=tokenizer)

In [ ]:
# retriever = VectorIndexRetriever(
#   index=index,
#   similarity_top_k=3,
# )
# contexts = retriever.retrieve(question)
# contexts

In [ ]:
# contexts[0].node.id_

In [ ]:
# [] + ["b"]

In [ ]:
"""Sorry old guy"""
# import re

# def get_context(question_list):
#   if len(question_list) <= 2:
#       question = question_list[0]

#       retriever = VectorIndexRetriever(
#       index=index,
#       similarity_top_k=3,
#       )
#       contexts = retriever.retrieve(question)
#       context_list = [n.get_content() for n in contexts]
#       context = "\n\n".join(context_list + [question])

#   else:
#     context_list_old = []
#     retriever = VectorIndexRetriever(
#       index=index,
#       similarity_top_k=1,
#       )
#     for i in range(len(question_list)-1):
#       question = question_list[i]
#       contexts = retriever.retrieve(question)
#       context_list = context_list_old + [n.get_content() for n in contexts]
#       context_list_old = context_list

#     context = "\n\n".join(context_list + [question])
#   print(context)
#   return context

# ## Uncomment this for new checkpoints
# def get_prompt(question):
#     INSTRUCTION_PROMPT_TEMPLATE = """\
#     ### System:
#     벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

#     ### User:
#     <보고서>
#     {CONTEXT}
#     </보고서>
#     지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
#     <질문>
#     {QUESTION}
#     </질문>

#     ### Assistant:
#     """
#     RESPONSE_TEMPLATE = """\
#     {ANSWER}

#     """

#     question_list = re.split('[?!.]',question.lstrip().rstrip())
#     context = get_context(question_list)

#     return INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)


'Sorry old guy'

In [ ]:
# ## Uncomment this for new checkpoints
# def get_prompt(question, prev_q):
#     # prev_q is a must needed for some questions like this: What is the biggest cause of plaster revision? And please tell me how to solve this.”
#     INSTRUCTION_PROMPT_TEMPLATE = """\
#     ### System:
#     벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

#     ### User:
#     <보고서>
#     {CONTEXT}
#     </보고서>
#     지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
#     <질문>
#     {QUESTION}
#     </질문>

#     ### Assistant:
#     """
#     RESPONSE_TEMPLATE = """\
#     {ANSWER}

#     """

#     retriever = VectorIndexRetriever(
#       index=index,
#       similarity_top_k=2,
#     )
#     contexts = retriever.retrieve(question)
#     context_list = [n.get_content() for n in contexts]
#     context = prev_q + "\n\n".join(context_list + [question])

#     return INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)

# # def get_prompt(question):
# #     INSTRUCTION_PROMPT_TEMPLATE = """\
# #     ### System:
# #     벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

# #     ### User:
# #     <보고서>
# #     {CONTEXT}
# #     </보고서>
# #     지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
# #     <질문>
# #     {QUESTION}
# #     </질문>

# #     """
# #     RESPONSE_TEMPLATE = """\
# #     ### Assistant:
# #     {ANSWER}

# #     """

# #     retriever = VectorIndexRetriever(
# #       index=index,
# #       similarity_top_k=2,
# #     )
# #     contexts = retriever.retrieve(question)
# #     context_list = [n.get_content() for n in contexts]
# #     context = "\n\n".join(context_list + [question])

# #     return INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)

In [ ]:
get_prompt("원목사 이딩의 단점은  주로  가격대가  상대적으로  비싸 고 관리 가 어렵다는  것입니다. 또한  습기에  노출될 경우 곰팡이와  곰 이 냄 새가 발생하여  건물 내부의  환경 을 손 상시킬  수 있습니다. 따라 서 원목사 이딩을 사용 할 때에는  이러한 단점을  고려하여  적절한  관리 가 필요합니다.")

'    ### System:\n    벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n    ### User:\n    <보고서>\n    마감재, 원목사이딩을 사용하는데 어떤 단점이 있을까요? 원목사이딩의 단점이 뭐야?, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다. 원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다. 원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다.\n\n마감재, 원목사이딩을 사용하는 것이 좋은 이유가 뭔가요? 원목사이딩의 장점이 뭐야?, 원목사이딩의 장점은 단열이 우수하고 외장재 중 가장 친환경적이며시공이 비교적 쉽다는 점입니다. 원목사이딩의 또 다른 장점은 내구성이 매우 우수하다는 것입니다. 원목사이딩은 가장 자연스럽고 아름다운 외관을 가지고 있으며, 오랜 기간 동안 매력적으로 유지됩니다. 게다가 자연재료로 만들어져 친환경적이며, 단열성능도 우수하여 에너지 비용을 절약할 수 있습니다. 시공도 비교적 쉽고 유지보수 비용 또한 적게 들어가는 편입니다. 원목사이딩의 장점은 주로 단열성능이 우수하며, 외장재 중에서 가장 친환경적인 재료로 시공이 비교적 쉽다는 점입니다. 또한 자연스러운 우드 텍스처와 색감을 살려 건물에 따뜻하고 자연스러운 분위기를 부여할 수 있습니다. 원목사이딩의 장점은 주택이나 

In [ ]:
# sample = "### System:\n벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n### User:\n<보고서>\n면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다. 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다. 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.\n</보고서>\n지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.\n<질문>\n면진장치가 뭐야? 면진장치에 사용되는 주요 기술은 무엇인가요?\n</질문>\n\n"
QUESTION = "방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?"
QUESTION = "또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?"
prompt_sample = get_prompt(QUESTION)

print(prompt_sample)
# outputs = text_pipe(prompt_sample, max_new_tokens=256, temperature=0.1, top_k=50, top_p=0.1, return_full_text=False)
outputs = text_pipe(prompt_sample,
                    min_new_tokens=20,
                    max_new_tokens=256,
                    top_p=0.98,
                    top_k=50,
                    temperature=0.9,
                    return_full_text=False,
                    eos_token_id = [27332]
                    )

print(outputs)
outputs[0].get('generated_text')

    ### System:
    벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

    ### User:
    <보고서>
    마감재, 원목사이딩을 사용하는데 어떤 단점이 있을까요? 원목사이딩의 단점이 뭐야?, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인한 변형 가능성 등이 있습니다. 또한 너무 많은 습기에 노출될 경우 곰팡이와 Decayed Wood와 같은 문제가 발생할 수 있습니다. 원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다. 원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다.

또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
    </보고서>
    지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
    <질문>
    또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
    </질문>

    ### Assistant:
    


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.98` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[{'generated_text': '마감재와 원목사 이딩의 단점은  주로  비용 이 상대적으로  높 고 관리 가 어렵다는  점입니다. 또한  습기에  노출될 경우 곰팡이와  곰팡 이 냄 새가 발생 할 수 있어  유지보수 가 어려울 수 있습니다.\n\n'}]


'마감재와 원목사 이딩의 단점은  주로  비용 이 상대적으로  높 고 관리 가 어렵다는  점입니다. 또한  습기에  노출될 경우 곰팡이와  곰팡 이 냄 새가 발생 할 수 있어  유지보수 가 어려울 수 있습니다.\n\n'

면진장치란  건물의  지반과 건물을  분리시키기  위해  면진 장치 를 사용하는  구조 를 말합니다. 이 구조 는 지반으로부터  오는  진동 에너지 를 흡 수하여  건물에  오는  진동 격리장치입니다. 이를  통해 건물 내부의  안정성을  유지하고  품질관리 가 가능하며, 원가절감과  빠 른 대량생산 이 가능합니다.

  System:
벽지에  대한  고객  문의 에 정확하고  유용한 답변을  작성한 다. <질문>의 의도를  파악하여  정확하게  <보고서 >만을  기반으로  답변하세요.

  User:
<보고서 >
도배풀 은 도배풀 에 본드가  섞 여 만든 풀 을 의미하며  내구성 이 좋고 내수성이  좋다는  장점이  있지 만 비용 이 다소 비싸

In [ ]:
import pandas as pd

test = pd.read_csv('test.csv')

In [ ]:
test.질문

0      방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용...
1                도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2        큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
3      철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있...
4                       도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
                             ...                        
125    분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126                            압출법 보온판의 가장 큰 장점은 무엇인가요?"
127    평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128    석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...
129       카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
Name: 질문, Length: 130, dtype: object

In [ ]:
import re
from tqdm import tqdm

for idx, question in tqdm(enumerate(test.질문)):
    questions = question.split("?!.")
    # print(len(questions))
    if len(questions)>2:
      print(questions)
    # print(re.split('[?!.]',question))


130it [00:00, 458201.28it/s]


In [ ]:
tokenizer.encode("###")

[1, 27332]

In [ ]:
# embed_model.encode("방청 페인트의 기능이 뭐야?", return_dense=True, return_sparse=True, return_colbert_vecs=False)


In [ ]:
"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?".split("?!.")

['방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?']

In [ ]:
for q in re.split('[?!.]', "방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?"):
  print(len(q) > 2)

True
True
False


In [ ]:
len(test)

130

In [ ]:
from tqdm import tqdm
import re

responses = []

for idx, question in tqdm(enumerate(test.질문)):
    # output = text_pipe(prompt_sample, max_new_tokens=256, temperature=0.1, top_k=50, top_p=0.1, return_full_text=False)
    # output = text_pipe(prompt_sample, max_new_tokens=256, num_beams=3, length_penalty = 1.1, return_full_text=False)


    # print(question)
    questions = re.split('[?!.]', question)
    seperate_output = []
    prev_q = ""
    for q in questions:
      if len(q) <= 2:
        continue

      prompt_sample = get_prompt(q + "?", prev_q=prev_q)
      output = text_pipe(prompt_sample,
                          min_new_tokens=20,
                          max_new_tokens=256,
                          top_p=0.98,
                          top_k=50,
                          temperature=0.9,
                          return_full_text=False,
                          eos_token_id = [27332]
                          )
      output = output[0].get('generated_text').lstrip().rstrip()
      seperate_output.append(output)
      prev_q = q

    answer = " ".join(seperate_output)
    # print(answer)

    responses.append(answer)

    # if idx > 1:
    #     break
responses


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.98` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
6it [02:47, 24.86s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
56it [33:22, 35.77s/it]


ValueError: Calculated available context size -62 was not non-negative.

In [ ]:
len(responses)

In [ ]:
for idx, question in tqdm(enumerate(test.질문)):
    print(question)
    print(responses[idx])
    print("***********")


In [ ]:
import json

json_data = json.dumps(responses)

# Write the JSON data to a file
with open('./good_responses_v3.json', 'w') as json_file:
    json_file.write(json_data)

In [ ]:
import json

# Open the JSON file in read mode
with open('good_responses_v3.json', 'r') as json_file:
    # Read the contents of the file
    json_data = json_file.read()

    # Parse the JSON data back into a Python data structure
    answer = json.loads(json_data)

len(answer)

In [ ]:
answer[:2]

In [ ]:
submission = pd.read_csv('sample_submission.csv')


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
test_embed_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

responses=answer
id = submission.id
submission.drop(['id'],axis = 1, inplace = True)



In [ ]:
for i in range(len(submission)):
    # print(i)
    # try:
    submission.loc[i] = test_embed_model.encode(responses[i]).tolist()
    # except:
    #     print(i)

In [ ]:
total_submission = pd.DataFrame({'id': id})
total_submission

In [ ]:
submission

In [ ]:
sub = pd.concat([total_submission, submission] , axis = 1)


In [ ]:
sub.to_csv('fun_submission_v3.csv', index = False) #Rename


### Use Custom NodeParser to divide the documents into chunks such that each row in train.csv file is treated as a single chunk

In [ ]:
# Global settings
from llama_index.core import Settings
# Local settings
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader("/content/drive/MyDrive/DACON_wallpaper_contest_data/data").load_data()

"""Alternative quicker way to use SentenceSplitter with custom chunk_size but it doesn't allow us to view the contents of divided chunks directly"""
# index2 = VectorStoreIndex.from_documents(
#     documents,
#     transformations=[SentenceSplitter(chunk_size=1295, paragraph_separator="\n")],
#     insert_batch_size=2
# )

"Alternative quicker way to use SentenceSplitter with custom chunk_size but it doesn't allow us to view the contents of divided chunks directly"

In [ ]:
# This chunk size is chosen to match the exact number of rows, the end of each entry contains the line break "\n"
splitter = SentenceSplitter(chunk_size=1295, paragraph_separator="\n")
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
len(nodes) # matches exactly to the row of train.csv file

645

In [ ]:
nodes[0]

TextNode(id_='794e14a9-2580-4e29-b16b-69e2c4061153', embedding=None, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creation_date': '2024-03-03', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9d1b58e2-cc97-453a-8a4a-03ba978b8451', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creatio

In [ ]:
nodes[1]

TextNode(id_='ef922520-07f9-4522-879b-c7aaa8861735', embedding=None, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creation_date': '2024-03-03', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9d1b58e2-cc97-453a-8a4a-03ba978b8451', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creatio

In [ ]:
print(nodes[621].get_content())

TRAIN_620, 내 침실을 더 현대적으로 꾸미고 싶어요. 어떤 컬러 팔레트를 추천하시나요?, 침실을 더 현대적으로 꾸미고 싶은데, 어떤 컬러 팔레트를 고려해야 할까요?, 인테리어, 침실을 현대적으로 꾸미려면 중립 컬러와 강조하는 팝 컬러를 혼합하여 시각적인 다양성을 만들어보세요., 침실을 현대적으로 꾸미기 위해서는 중립 컬러와 강조하는 팝 컬러를 혼합하여 시각적으로 다채로운 분위기를 연출해보는 것이 좋습니다. 예를 들어, 베이지, 회색 또는 화이트와 같은 중립 컬러를 베이스로 활용하면 부드러운 느낌을 연출할 수 있고, 강하고 선명한 컬러인 청록, 핑크 또는 코랄과 같은 팝 컬러를 조화롭게 함께 활용하면 세련된 분위기를 연출할 수 있습니다. 이렇게 중립 컬러와 팝 컬러를 조화롭게 결합하여 공간에 균형을 더해보세요., 침실을 더 현대적으로 꾸미고 싶다면 중립 컬러와 강조하는 팝 컬러를 혼합하여 사용하는 것을 추천합니다. 중립 컬러로는 베이지, 회색, 화이트와 같은 색상을 선택하여 고급스러운 느낌을 더할 수 있고, 강조하는 팝 컬러는 청록, 코랄 핑크, 민트 그린과 같이 생기 넘치는 밝은 컬러를 활용하여 분위기를 전환할 수 있습니다. 이러한 색상 조합을 통해 고요하면서도 활력 넘치는 분위기를 조성할 수 있을 것입니다., 침실을 현대적으로 꾸미려면 중립 컬러와 강조하는 팝 컬러를 혼합하여 시각적인 다양성을 만들어보세요. 예를 들어, 베이지, 회색 또는 흰색과 같은 중립 컬러를 베이스로 활용하고, 팝 컬러로는 청록, 산호색 또는 목련색과 같은 밝고 생동감 넘치는 컬러를 활용해보시는 것을 추천드립니다. 이러한 컬러 팔레트는 현대적이면서도 조화로운 분위기를 조성할 수 있습니다., 침실을 현대적으로 꾸미고 싶다면 중립 컬러와 함께 강조하는 팝 컬러를 활용하는 것이 좋습니다. 예를 들어, 베이지, 회색 또는 화이트와 같은 중립 컬러를 베이스로 사용하면 공간을 확장하고 안정감을 줄 수 있습니다. 그리고 강렬한 빨강, 푸른색 또는 황록색과 같은 팝 컬러를 가구, 소품 또는 벽

In [ ]:
index_chunk_size_1295 = VectorStoreIndex(nodes=nodes, insert_batch_size=2)

In [ ]:
PERSIST_DIR = "/content/drive/MyDrive/DACON_wallpaper_contest_data/train-vector-index-storage-chunk-size-1295"
index_chunk_size_1295.storage_context.persist(persist_dir=PERSIST_DIR)


### To do: Make different nodes for each of train.csv and other reference files from competition 1 (img to text), reference papers and korean wikipedia. Then make a combined index using VectorStoreIndex(nodes=[nodes1, nodes2])

### [Experiment] SentenceWindowNodeParser and JSONNodeParser

In [ ]:
from llama_index.core.node_parser import SentenceWindowNodeParser, TokenTextSplitter

window_node_parser = SentenceWindowNodeParser(
    window_size=4,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    # sentence_splitter = TokenTextSplitter()
)

window_nodes = window_node_parser.get_nodes_from_documents(documents)


In [ ]:
window_nodes[0]

TextNode(id_='8d94c89a-a0f7-4b21-9ee2-80c085b670a2', embedding=None, metadata={'window': 'TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다.  주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다.  이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다.  지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다.  이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. ', 'original_text': 'TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. ', 'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': '

In [ ]:
# Strange error occurs that the passed document has no id, if I pass the sentence_splitter.
# But checking as below show that document id is contained inside the document. Maybe a bug in llamaindex node parser.
# SentenceWindowNodeParser is an experiment so not much of priority as of now.
for doc in documents:
  print(doc.id_)

9d1b58e2-cc97-453a-8a4a-03ba978b8451


In [ ]:
from llama_index.core.node_parser import JSONNodeParser

json_splitter = JSONNodeParser()
nodes2 = json_splitter.get_nodes_from_documents(documents)
nodes2

[]

In [ ]:
"Doesn't work"
# from llama_index.core.node_parser import SimpleFileNodeParser

# file_type_splitter = SimpleFileNodeParser()
# nodes2 = file_type_splitter.get_nodes_from_documents(documents)
# nodes2


"Doesn't work"

In [ ]:
# nodes+window_nodes

In [ ]:
testing_combining_different_nodes = VectorStoreIndex(nodes=nodes+window_nodes, insert_batch_size=2)

KeyboardInterrupt: 